In [11]:
import pandas as pd
import cvxpy as cp
import numpy as np

In [12]:
# import data from pricing - Sheet2.csv into a pandas dataframe
df = pd.read_csv('pricing - Sheet2-2.csv')
#import optimization as opt

In [13]:
def modify_input(input):
    # Expects a list of movies or tv shows with the following inputs:
    # title, release date, image, streaming providers, media type
    providers = {}
    streaming_services = []
    prices = {}
    for item in input:
        title = item["title"]
        providers[title] = []
        services = item["streaming_providers"]
        for service in services["flatrate"]:
            # check if service is in first column of df
            index = df.index[df["Name"] == service].tolist()
            # If the service exists
            if index:
                location = index[0] # Find location
                service_info = df.loc[location, :] # Get pandas object of the row information
                if service_info["Name"] not in streaming_services:
                    streaming_services.append(service_info["Name"])
                if service_info["Price"] == '-': # If the service doesn't have a price
                    # Check if it is an add on
                    if service_info["Add-On"] == 1:
                         # If it is an add on, check if the base service is in the list of providers
                         if not service_info["Where"] == "None":
                                # If the base service is in the list of providers, add the price of the base service to the price of the add on
                                base_service = df.index[df["Name"] == service_info["Where"]].tolist()[0]
                                price = df.loc[base_service, :]["Price"]
                                prices[service] = float(price)
                else: # If it is not an add on, we can directly append the price
                    prices[service] = float(service_info["Price"])
            if service not in providers[title]:
                providers[title].append(service)
    streaming_services_out = []
    for streaming_service in streaming_services:
        if streaming_service in prices.keys():
            streaming_services_out.append(streaming_service)
    for key in providers.keys():
        temp = []
        for service in providers[key]:
            if service in streaming_services_out:
                temp.append(service)
        providers[key] = temp
    tobedeleted = [key for key in providers.keys() if len(providers[key]) == 0]
    for key in tobedeleted:
        print("We are sorry, but " + key + " is not available on any of the streaming services you have selected.")
        del providers[key]
    return providers, prices, streaming_services_out

In [14]:

search_results = [
    {
        "title": "Star Wars",
        "release_date": "1977-05-25",
        "image": "/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg",
        "rating": 8.2,
        "streaming_providers": {
            "flatrate": [
                "Disney Plus",
                "Sling TV Orange and Blue",
                "TNT",
                "TBS",
                "tru TV"
            ],
            "rent": [
                "Amazon Video",
                "Google Play Movies",
                "YouTube",
                "Redbox",
                "DIRECTV",
                "AMC on Demand",
                "Spectrum On Demand"
            ],
            "buy": [
                "Apple TV",
                "Amazon Video",
                "Google Play Movies",
                "YouTube",
                "Vudu",
                "Microsoft Store",
                "Redbox",
                "DIRECTV",
                "AMC on Demand"
            ]
        }
    },  
     {
        "title": "Star Wars: Andor",
        "release_date": "2022-09-21",
        "image": "/59SVNwLfoMnZPPB6ukW6dlPxAdI.jpg",
        "rating": 8.239,
        "streaming_providers": {
            "flatrate": [
                "Disney Plus"
            ],
            "rent": [],
            "buy": []
        }
    }, 
{
        "title": "Marvel's Avengers",
        "release_date": "2013-05-26",
        "image": "/vchDkX1DtqTy3bIDJ7YqmSbX965.jpg",
        "rating": 7.631,
        "streaming_providers": {
            "flatrate": [
                "Disney Plus",
                "DIRECTV"
            ],
            "rent": [],
            "buy": [
                "Apple TV",
                "Amazon Video",
                "Google Play Movies",
                "Vudu"
            ]
        }
    },
    {
        "title": "Star Wars: The Clone Wars",
        "release_date": "2008-08-05",
        "image": "/d6YOfi0T9GowglzIkDQGGvGYVTM.jpg",
        "rating": 6.1,
        "streaming_providers": {
            "flatrate": [
                "Disney Plus"
            ],
            "rent": [],
            "buy": []
        }
    },
    20.00 
]
budget = search_results[-1]
search_results = search_results[:-1]
providers, prices, services = modify_input(search_results)
print(providers)
print(prices)
print(services)
print(budget)

{'Star Wars': ['Disney Plus', 'Sling TV Orange and Blue'], 'Star Wars: Andor': ['Disney Plus'], "Marvel's Avengers": ['Disney Plus', 'DIRECTV'], 'Star Wars: The Clone Wars': ['Disney Plus']}
{'Disney Plus': 8.0, 'Sling TV Orange and Blue': 60.0, 'DIRECTV': 64.99}
['Disney Plus', 'Sling TV Orange and Blue', 'DIRECTV']
20.0


In [16]:
watchlist = list(providers.keys())
x = {m: cp.Variable(boolean=True) for m in watchlist}
y = {s: cp.Variable(boolean=True) for s in services}

objective = cp.sum(1 - cp.vstack([x[m] for m in watchlist])) + cp.sum(cp.vstack([y[s] for s in services]))

#services = list(set([s for v in providers.values() for s in v]))

constraints = [
    cp.sum(cp.vstack([y[s] * prices[s] for s in services])) <= budget,
]
for m in providers:
    print(m)
    constraints += [x[m] <= cp.sum(cp.vstack([y[s] for s in providers[m]]))]
    for s in services:
        if s in providers[m]:
            constraints += [y[s] <= cp.sum(cp.vstack([x[m] for m in providers]))]
    
problem = cp.Problem(cp.Minimize(objective), constraints)
problem.solve()

watch_opt = []
services_opt = []
print("Movies included in the package:")
for m in watchlist:
    if x[m].value == 1:
        print("- ", m)
        watch_opt.append(m)

print("Streaming services included in the package:")
for s in services:
    if y[s].value == 1:
        print("- ", s)
        services_opt.append(s)


Star Wars
Star Wars: Andor
Marvel's Avengers
Star Wars: The Clone Wars
Movies included in the package:
-  Star Wars
-  Star Wars: Andor
-  Marvel's Avengers
-  Star Wars: The Clone Wars
Streaming services included in the package:
-  Disney Plus
